for each driver, creates a list of standard routes in that order so that the higher in the list a standard route is, the least the diversion of the driver will be, and 
the output of the program is: 

a file called driver.json that has for each driver, the 5 standard routes routes that if the driver does them, it minimizes the diversion. You can test this by considering as pool of standard routes those that originally the company has and also those that you recommend in the recStandard.json. The file driver.json has the following syntax:
[
	{driver:C, routes:[s10, s20, s2, s6, s10}}, 
	{driver:A, routes:[s1, s2, s22, s61, s102]}, 
….
]


In [79]:
import os
HOME = os.getcwd()
print('HOME: ',HOME)

import math
import json
import random
import pandas as pd
import sys
import lxml
import sklearn as sk
import numpy as np

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import HDBSCAN
from scipy.spatial.distance import cosine

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go


HOME:  c:\Users\matti\Desktop\CODE\DataMiningProject23-24


In [80]:
STANDARD_FILE = 'standard_small.json'
ACTUAL_FILE = 'actual_small.json'

In [81]:
# load standard and actual data
with open(os.path.join('data',STANDARD_FILE)) as f:
    standard = json.load(f)

with open(os.path.join('data', ACTUAL_FILE)) as f:
    actual = json.load(f)

# load the data into a dataframe
dfStandard = pd.DataFrame(standard)
dfActual = pd.DataFrame(actual)

# print head of the dataframes
print(dfStandard.head())
print(dfActual.head())

# get the unique cities and items of the standard data
cities = []
items = []
drivers = []
longestRoute = 0
shortestRoute = np.inf
maxItemQuantity = 0
for index, s in dfStandard.iterrows():
    #print(s)
    idS = s['id']
    route = s['route']
    for trip in route:
        cities.append(trip['from']) 
        items.extend(trip['merchandise'].keys())
        maxItemQuantity = max(maxItemQuantity, max(trip['merchandise'].values()))
    if len(route) > 0:
        cities.append(route[-1]['to'])
        
    if len(route) > longestRoute:
        longestRoute = len(route)
        
    if len(route) < shortestRoute:
        shortestRoute = len(route)

for index, s in dfActual.iterrows():
    #print(s)
    idS = s['id']
    route = s['route']
    drivers.append(s['driver'])
    for trip in route:
        cities.append(trip['from'])
        items.extend(trip['merchandise'].keys())
        maxItemQuantity = max(maxItemQuantity, max(trip['merchandise'].values()))
        
    if len(route) > 0:
        cities.append(route[-1]['to'])
        
    if len(route) > longestRoute:
        longestRoute = len(route)
    
    if len(route) < shortestRoute:
        shortestRoute = len(route)

# find the unique cities and items
uniqueCities = sorted(list(set(cities)))
#uniqueCities.insert(0, 'NULL')          # add NULL city, for padding vectors with different lengths (trips in routes)
uniqueItems = sorted(list(set(items)))
uniqueDrivers = sorted(list(set(drivers)))

threeShingles = []

for i, c1 in enumerate(uniqueCities):
    for j, c2 in enumerate(uniqueCities):
        if i == j:
            continue
        for k, c3 in enumerate(uniqueCities):
            if j == k or i == k:
                continue
            threeShingles.append([c1, c2, c3])
            
permutations = math.perm(len(uniqueCities), 3)

print("Unique cities: ", uniqueCities)
print("Unique items: ", uniqueItems)
print("Unique drivers: ", uniqueDrivers)

print("Number of cities: ", len(uniqueCities))
print("Number of items: ", len(uniqueItems))
print("Number of drivers: ", len(uniqueDrivers))

print("\nLongest route: ", longestRoute)
print("Shortest route: ", shortestRoute)

print("\nMax item quantity: ", maxItemQuantity)

print("\nNumber of three-shingles: ", len(threeShingles))

print("\nThree-shingles: ", math.perm(len(uniqueCities), 3))
print("Three-shingles: ", math.comb(len(uniqueCities), 3))


standardIds = dfStandard['id'].tolist()
print("\nstandardIds: ", standardIds)
print("standardIds shape: ", np.array(standardIds).shape)
print("standardIds type: ", type(standardIds))


   id                                              route
0  s0  [{'from': 'Syracuse', 'to': 'Grosseto', 'merch...
1  s1  [{'from': 'Potenza', 'to': 'Cava de' Tirreni',...
2  s2  [{'from': 'Latina', 'to': 'Grosseto', 'merchan...
3  s3  [{'from': 'Syracuse', 'to': 'Lamezia Terme', '...
4  s4  [{'from': 'Cagliari', 'to': 'Verona', 'merchan...
   id driver sroute                                              route
0  a0      D     s0  [{'from': 'Syracuse', 'to': 'Grosseto', 'merch...
1  a1      G     s0  [{'from': 'Syracuse', 'to': 'Grosseto', 'merch...
2  a2      B     s0  [{'from': 'Syracuse', 'to': 'Grosseto', 'merch...
3  a3      B     s0  [{'from': 'Syracuse', 'to': 'Montesilvano', 'm...
4  a4      D     s0  [{'from': 'Grosseto', 'to': 'Pescara', 'mercha...
Unique cities:  ['Ancona', 'Andria', 'Anzio', 'Bisceglie', 'Bologna', 'Bolzano-Bozen', 'Cagliari', 'Caserta', 'Catania', "Cava de' Tirreni", 'Civitavecchia', 'Crotone', 'Ercolano', 'Ferrara', 'Fiumicino', 'Grosseto', 'La Spezia', 'L

In [82]:
def hashShingles(shingles, n):
    # hash shingles
    string = "" 
    for shingle in shingles:
        string += str(shingle) + "," # [45, 4, 8] -> "45,4,8,"
    
    return hash(string) #% n

def createShingles(df, k, uniqueCities, uniqueItems, longestRoute, maxItemQuantity, permutations):
    # create shingles for each route
    shingles = []
    for index, s in df.iterrows():
        idS = s['id']
        route = s['route']
        shingle = [index]
        citiesInRoute = [] # napoli roma milano teramo bergamo [10,4,5,48,12] [10,4,5] [4,5,48] [5,48,12]
        merchandiseInRoute = np.zeros(len(uniqueItems))
        for trip in route:
            citiesInRoute.append(uniqueCities.index(trip['from']))
            #merchandiseInRoute += np.array(list(trip['merchandise'].values()))
            for item, n in trip['merchandise'].items():
                merchandiseInRoute[uniqueItems.index(item)] += n
        if len(route) > 0:
            citiesInRoute.append(uniqueCities.index(route[-1]['to']))
        merchandiseInRoute = np.array(merchandiseInRoute) / (maxItemQuantity*len(route))
        
        hashedShingles = []
        for i in range(len(citiesInRoute)-k+1):
            # Q: is it correct to set the modulo for the hash function to the number of permutations?
            # A: yes, because we want to have a unique hash for each shingle
            # Q: would it be better to use a different hash function?
            # A: yes, because the modulo function is not a good hash function
            hashedShingles.append(hashShingles(citiesInRoute[i:i+k], permutations) )
        
        shingle.append(np.array(hashedShingles))
        
        shingle.append(merchandiseInRoute) # quantity hot encoding
        
        shingles.append(shingle)
        
    return shingles # [ index, [shingles], [merchandise] ]

In [83]:
standardSets = createShingles(dfStandard, k=3, uniqueCities=uniqueCities, uniqueItems=uniqueItems, longestRoute=longestRoute, maxItemQuantity=maxItemQuantity, permutations=permutations)
actualSets = createShingles(dfActual, k=3, uniqueCities=uniqueCities, uniqueItems=uniqueItems, longestRoute=longestRoute, maxItemQuantity=maxItemQuantity, permutations=permutations)

print("\nstandardSets", len(standardSets), "shape first element", standardSets[0][1].shape, standardSets[0])
print("\nactualSets", len(actualSets),  "shape first element", standardSets[0][1].shape, actualSets[0])

print("\nstandardSets:", len(standardSets))
print("actualSets:", len(actualSets))




assert len(standardSets[0]) == 3, "The length of the standard set is not equal to 3 (index, shingles, merchandise)"
assert len(standardSets[0][2]) == len(uniqueItems), "The length of the merchandise vector is not equal to the number of unique items"


standardSets 10 shape first element (27,) [0, array([ 1816899795994065000,  6859925411632190335, -6996249020691994405,
        9168582141423881541,  8005217413185281646, -3922973237653836308,
        7857788239845868483, -6888207344237923958,   557676716925075772,
       -4387038302106863052,  2356827645895490470, -1714745471101618149,
        4454129151350282811,  -277120799498575755,  3041468598048882567,
       -7030291844422291893, -1421986576509754467, -4062136624691345276,
        2417343010238685400, -3430512250377713180, -8314828373687969508,
       -6821424956212609228,  8169809827005466252,  3951088186650106639,
        -739580167742056683,  8451748510638040908,  3276108207156170051],
      dtype=int64), array([0.26904762, 0.27619048, 0.27619048, 0.30952381, 0.15119048,
       0.2452381 , 0.25833333, 0.24166667, 0.31785714, 0.16190476])]

actualSets 1000 shape first element (27,) [0, array([ 1816899795994065000,  6859925411632190335, -6996249020691994405,
        83194424179

In [84]:
def jaccard_similarity_matrix(matrix):
    intersection = np.dot(matrix, matrix.T)
    row_sums = matrix.sum(axis=1)
    union = row_sums[:, None] + row_sums - intersection
    union = np.where(union == 0, 1, union)  # avoid division by zero
    jaccard_similarity = intersection / union
    return jaccard_similarity

def jaccard_similarity_two_matrices(matrix1, matrix2):
    #intersection = np.dot(matrix, matrix.T)
    intersection = np.dot(matrix1, matrix2.T)
    row_sums1 = matrix1.sum(axis=1)
    row_sums2 = matrix2.sum(axis=1)
    union = row_sums1[:, None] + row_sums2 - intersection
    union = np.where(union == 0, 1, union)  # avoid division by zero
    jaccard_similarity = intersection / union
    return jaccard_similarity

In [85]:
def jaccard_similarity_matrix_merch(matrix):
    print("matrix", matrix.shape)
    min_matrix = np.minimum(matrix[:, None, :], matrix[None, :, :])
    sum_min_matrix = np.sum(min_matrix, axis=-1)
    print("sum_min_matrix", sum_min_matrix.shape)
    
    max_matrix = np.maximum(matrix[:, None, :], matrix[None, :, :])
    sum_max_matrix = np.sum(max_matrix, axis=-1)
    print("sum_max_matrix", sum_max_matrix.shape)
    
    jaccard_similarity = sum_min_matrix / sum_max_matrix
    return jaccard_similarity

def jaccard_similarity_matrices_merch(matrix1, matrix2):
    print("matrix1", matrix1.shape)
    print("matrix2", matrix2.shape)
    
    min_matrix = np.minimum(matrix1[:, None, :], matrix2[None, :, :])
    sum_min_matrix = np.sum(min_matrix, axis=-1)
    print("sum_min_matrix", sum_min_matrix.shape)
    
    max_matrix = np.maximum(matrix1[:, None, :], matrix2[None, :, :])
    sum_max_matrix = np.sum(max_matrix, axis=-1)
    print("sum_max_matrix", sum_max_matrix.shape)
    
    jaccard_similarity = sum_min_matrix / sum_max_matrix
    return jaccard_similarity


def create_binary_matrix(routeSets):
    # create binary matrix where each row represents a route
    uniqueShingles = list(set([shingle for route in routeSets for shingle in route[1]]))
    binaryMatrix = np.zeros((len(routeSets), len(uniqueShingles)))
    for i, route in enumerate(routeSets):
        for shingle in route[1]:
            binaryMatrix[i][uniqueShingles.index(shingle)] = 1
    return binaryMatrix

def create_binary_matrices(routeSet1, routeSet2):
    # create binary matrix where each row represents a route
    uniqueShinglesBoth = list(set([shingle for route in routeSet1 for shingle in route[1]] + [shingle for route in routeSet2 for shingle in route[1]]))
    binaryMatrix1 = np.zeros((len(routeSet1), len(uniqueShinglesBoth)))
    binaryMatrix2 = np.zeros((len(routeSet2), len(uniqueShinglesBoth)))
    for i, route in enumerate(routeSet1):
        for shingle in route[1]:
            binaryMatrix1[i][uniqueShinglesBoth.index(shingle)] = 1
            
    for i, route in enumerate(routeSet2):
        for shingle in route[1]:
            binaryMatrix2[i][uniqueShinglesBoth.index(shingle)] = 1
    return binaryMatrix1, binaryMatrix2

## FIRST IDEA:

In [86]:
# binary matrix where each row represents a route
route_actual_matrix, route_standard_matrix = create_binary_matrices(actualSets,standardSets)

# compute Jaccard similarity for routes
matrix_sim__route_actual_standard = jaccard_similarity_two_matrices(route_actual_matrix, route_standard_matrix)

print("matrix_sim_actual_standard", matrix_sim__route_actual_standard.shape)
print("matrix_sim_actual_standard[0]", matrix_sim__route_actual_standard[1])


# 
merch_actual_matrix = np.array([s[2] for s in actualSets])
merch_standard_matrix = np.array([s[2] for s in standardSets])

# compute Jaccard similarity for merchandise
matrix_sim_merch_actual_standard = jaccard_similarity_matrices_merch(merch_actual_matrix, merch_standard_matrix)



SetsSimilarities = (matrix_sim__route_actual_standard + matrix_sim_merch_actual_standard) / 2
SetsSimilarities=np.nan_to_num(SetsSimilarities, nan=0)

print("SetsSimilarities", SetsSimilarities.shape)
print("SetsSimilarities[0]", SetsSimilarities[0])

matrix_sim_actual_standard (1000, 10)
matrix_sim_actual_standard[0] [0.55882353 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
matrix1 (1000, 10)
matrix2 (10, 10)
sum_min_matrix (1000, 10)
sum_max_matrix (1000, 10)
SetsSimilarities (1000, 10)
SetsSimilarities[0] [0.68456993 0.35273923 0.37805824 0.33381119 0.38744375 0.38857347
 0.38418176 0.33580855 0.37879655 0.39177168]


In [87]:
max_value_index = np.argmax(SetsSimilarities, axis=1)
max_value = np.max(SetsSimilarities, axis=1)

# print("Index of the biggest value for each row:", max_value_index)
# print("Value of the biggest value for each row:", max_value)

# print("len(max_value_index)", len(max_value_index))
# print("len(max_value)", len(max_value))

# [index for index, s in dfActual.iterrows() if s['driver'] == driver]
# uniqueDrivers
driver_indices = {}

for i, s in dfActual.iterrows():
    driver = s['driver']
    
    # Check if the driver is already in the dictionary
    if driver in driver_indices:
        # If yes, append the index to the existing array
        driver_indices[driver].append(i)
    else:
        # If not, create a new array with the current index
        driver_indices[driver] = [i]

print("driver_indices", driver_indices)
print("len(driver_indices)", len(driver_indices))

driver_indices {'D': [0, 4, 23, 31, 33, 45, 52, 58, 63, 65, 78, 90, 121, 126, 131, 132, 134, 162, 174, 185, 192, 193, 212, 214, 222, 259, 268, 279, 297, 300, 305, 321, 332, 339, 342, 373, 375, 394, 395, 396, 406, 423, 425, 439, 450, 475, 477, 485, 487, 519, 522, 540, 563, 564, 583, 584, 629, 648, 649, 671, 672, 679, 687, 738, 740, 741, 743, 752, 753, 758, 760, 778, 787, 790, 793, 796, 803, 808, 832, 846, 850, 861, 893, 896, 914, 921, 927, 938, 952, 963, 973, 981, 986, 992], 'G': [1, 9, 18, 20, 36, 41, 81, 87, 88, 105, 110, 114, 122, 129, 139, 140, 144, 147, 160, 186, 198, 216, 218, 220, 230, 237, 239, 280, 307, 314, 323, 348, 353, 354, 355, 361, 367, 391, 392, 401, 408, 415, 429, 434, 446, 461, 468, 474, 502, 504, 508, 517, 518, 520, 534, 546, 561, 575, 592, 599, 605, 606, 613, 623, 634, 644, 645, 652, 662, 673, 678, 693, 704, 706, 711, 713, 725, 735, 779, 812, 820, 824, 827, 838, 841, 852, 866, 876, 884, 899, 900, 902, 904, 909, 910, 928, 932, 933, 977, 985, 988, 991, 995, 999], 'B': 

In [88]:
# Create a dictionary of all drivers' routes
drivers_routes = {}

for driver in uniqueDrivers:
    print("driver", driver)
    # print("driver_indices[driver]", driver_indices[driver])
    # print("len(driver_indices[driver])", len(driver_indices[driver]))

    driver_standard_index = max_value_index[driver_indices[driver]]
    driver_max_value = max_value[driver_indices[driver]]
    # print("driver_max_value_index", driver_standard_index)

    # Assuming driver_max_value_index is a NumPy array
    unique_values_index = np.unique(driver_standard_index)
    # print("unique_values", unique_values_index)


    # Calculate the mean for each unique value
    means = [np.mean(driver_max_value[driver_standard_index == idx]) for idx in unique_values_index]
    # print("means", means)

    best_route_Ids = []
    # Print the results for each driver
    for idx, mean in zip(unique_values_index, means):
        # print(f"Driver: {driver}, Unique Value: {idx}, Mean: {mean}")
        best_route_Ids.append([standardIds[idx], mean])

    # print("best_route_Ids", best_route_Ids)
    # Sort the routes by their mean
    best_route_Ids.sort(key=lambda x: x[1], reverse=True)
    # print("best_route_Ids", best_route_Ids)
    
    # Keep the top 5 routes
    top_5_routes = best_route_Ids[:5]
    # print("top_5_routes", top_5_routes)

    # Update the driver's routes in the dictionary
    drivers_routes[driver] = {'driver': driver, 'routes': [id for id,value in top_5_routes]}

# Convert the dictionary to a list for JSON serialization
result_list = list(drivers_routes.values())

# Write the result to driver.json
with open(os.path.join('results', 'driver.json'), 'w') as outfile:
    json.dump(result_list, outfile, indent=2)

print(f"JSON driver data has been written to data/driver.json")

driver A
driver B
driver C
driver D
driver E
driver F
driver G
driver H
driver I
driver J
JSON driver data has been written to data/driver.json


## ERIC's IDEA

In [89]:
# binary matrix where each row represents a route
route_actual_matrix, route_standard_matrix = create_binary_matrices(actualSets,standardSets)

# compute Jaccard similarity for routes
matrix_sim__route_actual_standard = jaccard_similarity_two_matrices(route_actual_matrix, route_standard_matrix)

print("matrix_sim_actual_standard", matrix_sim__route_actual_standard.shape)
print("matrix_sim_actual_standard[0]", matrix_sim__route_actual_standard[1])

# 
merch_actual_matrix = np.array([s[2] for s in actualSets])
merch_standard_matrix = np.array([s[2] for s in standardSets])

# compute Jaccard similarity for merchandise
matrix_sim_merch_actual_standard = jaccard_similarity_matrices_merch(merch_actual_matrix, merch_standard_matrix)



SetsSimilarities = (matrix_sim__route_actual_standard + matrix_sim_merch_actual_standard) / 2
SetsSimilarities=np.nan_to_num(SetsSimilarities, nan=0)

print("SetsSimilarities", SetsSimilarities.shape)
print("SetsSimilarities[0]", SetsSimilarities[0])

matrix_sim_actual_standard (1000, 10)
matrix_sim_actual_standard[0] [0.55882353 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
matrix1 (1000, 10)
matrix2 (10, 10)
sum_min_matrix (1000, 10)
sum_max_matrix (1000, 10)
SetsSimilarities (1000, 10)
SetsSimilarities[0] [0.68456993 0.35273923 0.37805824 0.33381119 0.38744375 0.38857347
 0.38418176 0.33580855 0.37879655 0.39177168]


In [93]:
# max_value_index = np.argmax(SetsSimilarities, axis=1)
# max_value = np.max(SetsSimilarities, axis=1)
# "sroute": "s0",

driver_indices = {}
driver_standard = {}

for i, s in dfActual.iterrows():
    driver = s['driver']
    route = s['sroute']
    
    # Check if the driver is already in the dictionary
    if driver in driver_indices:
        # If yes, append the index to the existing array
        driver_indices[driver].append(i)
        driver_standard[driver].append(route)
    else:
        # If not, create a new array with the current index
        driver_indices[driver] = [i]
        driver_standard[driver] = [route]

print("driver_indices", driver_indices)
print("len(driver_indices)", len(driver_indices))

print("driver_standard", driver_standard)
print("len(driver_standard)", len(driver_standard))

driver_indices {'D': [0, 4, 23, 31, 33, 45, 52, 58, 63, 65, 78, 90, 121, 126, 131, 132, 134, 162, 174, 185, 192, 193, 212, 214, 222, 259, 268, 279, 297, 300, 305, 321, 332, 339, 342, 373, 375, 394, 395, 396, 406, 423, 425, 439, 450, 475, 477, 485, 487, 519, 522, 540, 563, 564, 583, 584, 629, 648, 649, 671, 672, 679, 687, 738, 740, 741, 743, 752, 753, 758, 760, 778, 787, 790, 793, 796, 803, 808, 832, 846, 850, 861, 893, 896, 914, 921, 927, 938, 952, 963, 973, 981, 986, 992], 'G': [1, 9, 18, 20, 36, 41, 81, 87, 88, 105, 110, 114, 122, 129, 139, 140, 144, 147, 160, 186, 198, 216, 218, 220, 230, 237, 239, 280, 307, 314, 323, 348, 353, 354, 355, 361, 367, 391, 392, 401, 408, 415, 429, 434, 446, 461, 468, 474, 502, 504, 508, 517, 518, 520, 534, 546, 561, 575, 592, 599, 605, 606, 613, 623, 634, 644, 645, 652, 662, 673, 678, 693, 704, 706, 711, 713, 725, 735, 779, 812, 820, 824, 827, 838, 841, 852, 866, 876, 884, 899, 900, 902, 904, 909, 910, 928, 932, 933, 977, 985, 988, 991, 995, 999], 'B': 

In [110]:
# Create a dictionary of all drivers' routes
drivers_routes = {}

for driver in uniqueDrivers:
    print("driver", driver)
    print("driver_indices[driver]", driver_indices[driver])
    # print("len(driver_indices[driver])", len(driver_indices[driver]))
    print("driver_standard[driver]", driver_standard[driver])
    # print("len(driver_standard[driver])", len(driver_standard[driver]))

    driver_route_values = []
    
    for index, standard in zip(driver_indices[driver], driver_standard[driver]):
        driver_standard_value = SetsSimilarities[index][standardIds.index(standard)]
        driver_route_values.append(driver_standard_value)

    print("driver_route", driver_route_values)

    # Assuming driver_max_value_index is a NumPy array
    unique_values_index = np.unique(driver_standard[driver])
    print("unique_values", unique_values_index)

    # # Calculate the mean for each unique value
    means = [np.mean(driver_route_values[driver_standard[driver] == idx]) for idx in unique_values_index]
    print("means", means)

    best_route_Ids = []
    # Print the results for each driver
    for idx, mean in zip(unique_values_index, means):
        print(f"Driver: {driver}, Unique Value: {idx}, Mean: {mean}")
        best_route_Ids.append([idx, mean])

    # print("best_route_Ids", best_route_Ids)
    # Sort the routes by their mean
    best_route_Ids.sort(key=lambda x: x[1], reverse=True)
    # print("best_route_Ids", best_route_Ids)
    
    # Keep the top 5 routes
    top_5_routes = best_route_Ids[:5]
    # print("top_5_routes", top_5_routes)

    # Update the driver's routes in the dictionary
    drivers_routes[driver] = {'driver': driver, 'routes': [id for id,value in top_5_routes]}

# Convert the dictionary to a list for JSON serialization
result_list = list(drivers_routes.values())

# Write the result to driver.json
with open(os.path.join('results', 'driver2.json'), 'w') as outfile:
    json.dump(result_list, outfile, indent=2)

print(f"JSON driver data has been written to data/driver.json")

driver A
driver_indices[driver] [10, 19, 25, 43, 48, 50, 55, 64, 68, 94, 96, 106, 107, 113, 117, 128, 141, 172, 180, 191, 199, 213, 223, 227, 228, 244, 247, 251, 272, 281, 295, 303, 306, 308, 316, 340, 341, 345, 363, 366, 376, 378, 404, 418, 440, 469, 488, 496, 521, 548, 556, 559, 608, 610, 622, 627, 628, 630, 638, 656, 660, 718, 721, 731, 737, 746, 747, 749, 765, 780, 839, 853, 868, 871, 875, 878, 886, 890, 905, 919, 925, 958, 961, 969, 970, 989]
driver_standard[driver] ['s0', 's0', 's0', 's0', 's0', 's0', 's0', 's0', 's0', 's0', 's0', 's1', 's1', 's1', 's1', 's1', 's1', 's1', 's1', 's1', 's1', 's2', 's2', 's2', 's2', 's2', 's2', 's2', 's2', 's2', 's2', 's3', 's3', 's3', 's3', 's3', 's3', 's3', 's3', 's3', 's3', 's3', 's4', 's4', 's4', 's4', 's4', 's4', 's5', 's5', 's5', 's5', 's6', 's6', 's6', 's6', 's6', 's6', 's6', 's6', 's6', 's7', 's7', 's7', 's7', 's7', 's7', 's7', 's7', 's7', 's8', 's8', 's8', 's8', 's8', 's8', 's8', 's8', 's9', 's9', 's9', 's9', 's9', 's9', 's9', 's9']
driver_